In [2]:
from model import melody_ResNet
from model_torch import Melody_ResNet
import torch
import numpy as np

Using TensorFlow backend.


In [3]:
keras_model = melody_ResNet()
keras_model.load_weights('./weights/ResNet_NS.hdf5')


In [4]:
total_weights = []
for layer in keras_model.layers:
    weights = layer.get_weights()
    if weights != []:
        total_weights.append({'name': layer.name, 'weights': weights })

In [5]:
len(total_weights)

34

In [6]:
print([(i, x['name']) for i,x in enumerate(total_weights)])

[(0, 'conv_s1_1x1'), (1, 'batch_normalization_1'), (2, 'conv1_1'), (3, 'batch_normalization_2'), (4, 'conv1_2'), (5, 'batch_normalization_3'), (6, 'conv_f1_1x1'), (7, 'batch_normalization_4'), (8, 'conv_s2_1x1'), (9, 'batch_normalization_5'), (10, 'conv2_1'), (11, 'batch_normalization_6'), (12, 'conv2_2'), (13, 'batch_normalization_7'), (14, 'conv_f2_1x1'), (15, 'batch_normalization_8'), (16, 'conv_s3_1x1'), (17, 'batch_normalization_9'), (18, 'conv3_1'), (19, 'batch_normalization_10'), (20, 'conv3_2'), (21, 'batch_normalization_11'), (22, 'conv_f3_1x1'), (23, 'batch_normalization_12'), (24, 'conv_s4_1x1'), (25, 'batch_normalization_13'), (26, 'conv4_1'), (27, 'batch_normalization_14'), (28, 'conv4_2'), (29, 'batch_normalization_15'), (30, 'conv_f4_1x1'), (31, 'batch_normalization_16'), (32, 'bidirectional_1'), (33, 'time_distributed_1')]


In [9]:
torch_model = Melody_ResNet()

for i in range(4):
    block_weights = total_weights[i*8:(i+1)*8]
    for j in range(4):
        getattr(torch_model.block[i], f'conv{j+1}').conv.weight.data = torch.from_numpy(np.transpose(block_weights[j*2]['weights'][0], (3, 2, 0, 1)))
        getattr(torch_model.block[i], f'conv{j+1}').bn.weight.data = torch.from_numpy(block_weights[j*2+1]['weights'][0])
        getattr(torch_model.block[i], f'conv{j+1}').bn.bias.data = torch.from_numpy(block_weights[j*2+1]['weights'][1])
        getattr(torch_model.block[i], f'conv{j+1}').bn.running_mean.data = torch.from_numpy(block_weights[j*2+1]['weights'][2])
        getattr(torch_model.block[i], f'conv{j+1}').bn.running_var.data = torch.from_numpy(block_weights[j*2+1]['weights'][3])

lstm_weight_name = ['weight_ih_l0',
 'weight_hh_l0',
 'bias_ih_l0',
 'bias_hh_l0',
 'weight_ih_l0_reverse',
 'weight_hh_l0_reverse',
 'bias_ih_l0_reverse',
 'bias_hh_l0_reverse']
keras_lstm_index = [0,1,2,2,3,4,5,5]

for i, name in enumerate(lstm_weight_name):
    if i in (0,1,4,5):
        getattr(torch_model.lstm, name).data = torch.from_numpy(np.transpose(total_weights[-2]['weights'][keras_lstm_index[i]]))
    else:
        weight = total_weights[-2]['weights'][keras_lstm_index[i]]
        getattr(torch_model.lstm, name).data = torch.from_numpy(weight) / 2


torch_model.final.weight.data = torch.from_numpy(np.transpose(total_weights[-1]['weights'][0]))

torch_model.final.bias.data = torch.from_numpy(total_weights[-1]['weights'][1])

torch.save(torch_model.state_dict(), 'torch_weights.pt')


In [122]:
np.transpose(total_weights[4]['weights'][0]).shape

(64, 64, 3, 3)

In [91]:
state_dict['block.0.conv1.conv.weight'].shape

torch.Size([64, 1, 1, 1, 1])

In [5]:
# vars(torch_model.lstm)

# print([x.shape for x in total_weights[-2]['weights']])
# total_weights[-2]['weights'][0].shape
# lstm_weight_name

In [3]:
# vars(keras_model.layers[-3])

In [2]:
# vars(keras_model.layers[-3].forward_layer.cell)
